In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# path = '/global/homes/e/ecastori/PostBorn/'
path = '../PostBornEma/'

In [ ]:
data = np.loadtxt(path+'FFT_k-1Pk.dat',skiprows=2)

#----------------------#
n         = data[:,0]
Re_c_n    = data[:,1]
Im_c_n    = data[:,2]
Re_nu_n   = data[:,3]
Im_nu_n   = data[:,4]
#----------------------#

c_n  = Re_c_n  + 1j * Im_c_n
nu_n = Re_nu_n + 1j * Im_nu_n

In [ ]:
### When reading Ema's files for the first time

# data = np.loadtxt(path+'Il_nu_t_new1.dat',skiprows=1)
# # data = np.loadtxt(path+'Il_nu_t.dat',skiprows=1)

# #----------------------#
# ell       = data[:,0]
# nn        = data[:,1]
# tt        = data[:,2]
# Re_I      = data[:,3]
# Im_I      = data[:,4]
# #----------------------#

# I_ = Re_I + 1j * Im_I

# # np.save('../PostBornEma/Il_nu_t_new1-ell-nn-tt.dat', np.stack((ell, nn, tt), axis=1))
# # np.save('../PostBornEma/Il_nu_t_new1-ReI-ImI.dat', np.stack((Re_I, Im_I), axis=1))

In [ ]:
data =  np.load(path+'Il_nu_t_new1-ell-nn-tt.npy')
ell       = data[:,0]
nn        = data[:,1]
tt        = data[:,2]
#----------------------#

data =  np.load(path+'Il_nu_t_new1-ReI-ImI.npy')
Re_I      = data[:,0]
Im_I      = data[:,1]
#----------------------#

I_ = Re_I + 1j * Im_I

In [ ]:
data = np.loadtxt(path+'GG_weights.dat',skiprows=2)
t1   = data[:,0]
w1   = data[:,1]

In [ ]:
print(np.unique(tt)-t1)

In [ ]:
#cosmology
h         = 0.6770
omega_cdm = 0.11923
omega_b   = 0.02247
Omega_b   = omega_b/h**2
Omega_cdm = omega_cdm/h**2
Omega_m   = Omega_b+Omega_cdm
ns        = 0.96824
A0        = 2.10732*10**(-9)

#dictionary for class
cosmo_dict={'h': h,
'omega_b' : omega_b,
'omega_cdm': omega_cdm,
'A_s'    : A0,
'n_s'    : ns,
'k_pivot' : 0.05}
print(cosmo_dict)

# speed of light
c      = 299792458./1000. # km/s

#prefactor for Cl_kk computation from Cl_dd
prefac = 1.5*Omega_m*(100.)**2/c**2 #without h

In [ ]:
from classy import Class
cosmo = Class()
cosmo.set(cosmo_dict)
cosmo.compute()
cosmo_b               = cosmo.get_background()

class_z               = cosmo_b['z'][::-1]
class_chi             = cosmo_b['comov. dist.'][::-1]
class_D               = cosmo_b['gr.fac. D'][::-1]#/cosmo_b['gr.fac. D'][-1]

derivParams           = cosmo.get_current_derived_parameters(['z_rec'])
z_cmb                 = derivParams['z_rec']


from scipy.interpolate import interp1d
from scipy.interpolate import InterpolatedUnivariateSpline as ius
chi_z = interp1d(class_z,class_chi*h, bounds_error=False, fill_value=0)
z_chi = interp1d(class_chi*h,class_z, bounds_error=False, fill_value=0)   # Mpc/h
D_chi = interp1d(class_chi*h,class_D, bounds_error=False, fill_value=0)    # growth
D_z   = interp1d(class_z,class_D, bounds_error=False, fill_value=0)
# chi_cmb
chi_cmb = chi_z(z_cmb)



In [ ]:
print(chi_z(1100),chi_z(z_cmb))

In [ ]:
z = np.linspace(0,5,100)
plt.figure()
plt.plot(class_z,class_D,ls='-')
plt.plot(z,D_z(z),ls='--')
plt.plot(z,1/(1+z),ls='-.')
plt.xlim(0,5)
plt.show()
D_z(0)

In [ ]:
z=np.linspace(0.,10,200)
plt.figure()
plt.loglog(z,chi_z(z))
plt.show()
plt.figure()
plt.plot(z,D_z(z)/D_chi(chi_z(z)))
plt.show()
# plt.figure()
# plt.loglog(z,)
# plt.show()
# plt.figure()
# plt.loglog(z,z_chi(chi_z(z)))
# plt.show()

In [ ]:
D_chi(chi_z(0.1))

# Setup

In [ ]:
def set_kernel(chi_max):
    def kernel(chi):
        return (1.+z_chi(chi))*D_chi(chi)
    return kernel

In [ ]:
D = set_kernel(chi_cmb)

In [ ]:
ell

In [ ]:
ell_ = np.unique(ell)
nu_n_= np.unique(nu_n) 
t_=np.unique(tt)
assert(np.allclose(t_,t1))

print(nu_n_)
print(ell_)
print(np.unique(tt))
print(len(nu_n_),len(n),len(c_n),len(t_))
print(len(tt),len(I_))

In [ ]:
# m=0
# I_array=np.zeros((len(ell_),len(nu_n_),len(t_)),dtype=np.complex64)
# I2D = np.zeros((len(ell_),len(nu_n_),len(t_),len(t_)),dtype=np.complex64)
# for ii,ll in enumerate(ell_):
#     for jj, nu in enumerate(nu_n_):
#             #for kk, _ in enumerate(t_):
#         I_array[ii,jj,:]=I_[m:m+len(t_)]
#         _, I2D_ = np.meshgrid(t_,I_array[ii,jj])
#         I2D[ii][jj] = I2D_
#         m+=len(t_)
m=0
I2D = np.zeros((len(ell_),len(nu_n_),len(t_),len(t_)),dtype=np.complex64)
for ii,ll in enumerate(ell_):
    for jj, nu in enumerate(nu_n_):
        I2D[ii][jj] = np.meshgrid(t_,I_[m:m+len(t_)])[1]
        m+=len(t_)

# Integrals here

In [ ]:
from scipy.integrate import simps, quadrature
from scipy.interpolate import interp1d
import time


In [ ]:
r2d, t2d = np.meshgrid(t_,t_)
w11, w12 = np.meshgrid(w1,w1)
begin=time.time()
resultv=[]
for ii,ll in enumerate(ell_):
    if ii%20 ==0: print(ii)
    res1=[]
    for jj, nu in enumerate(nu_n_):
        #_, I2D = np.meshgrid(t_,I_array[ii,jj])
        integrand=(1.-r2d*t2d)/t2d*(1.-r2d)/r2d*I2D[ii][jj]*D(r2d*t2d*chi_cmb)*D(r2d*chi_cmb)*(r2d*chi_cmb)**(-nu)
        res1+=[np.sum(w11*w12*integrand)]
    resultv+=[np.real(np.sum(res1[0:-1]*c_n[0:-1]*2.)+res1[-1]*c_n[-1])]
print('Time taken = ', time.time()-begin)

Clv = np.real(np.asarray(resultv))*4./np.pi**2*prefac**2/4.


In [ ]:
###Introduce a general kernel here 

def Wkernel(xi, xmax):
    return (xmax - xi)/(xmax*xi) * (xmax > xi)

# def Wout(r, xmax):
#     x1 = r*xmax
#     return (xmax - x1)/(xmax*x1) * (xmax > x1)

# def Win(t, r, xmax):
#     x2 = r*t*xmax
#     return (xmax - x2)/(xmax*x2) * (xmax > x2)


r2d, t2d = np.meshgrid(t_,t_)
w11, w12 = np.meshgrid(w1,w1)
begin=time.time()
result=[]
for ii,ll in enumerate(ell_):
    if ii%20 ==0: print(ii)
    res1=[]
    for jj, nu in enumerate(nu_n_):
        #_, I2D = np.meshgrid(t_,I_array[ii,jj])
        ww = Wkernel(r2d*chi_cmb, chi_cmb) *Wkernel(r2d*t2d*chi_cmb, chi_cmb)
        integrand=ww*I2D[ii][jj]*D(r2d*t2d*chi_cmb)*D(r2d*chi_cmb)*(r2d*chi_cmb)**(1-nu)
        res1+=[np.sum(w11*w12*integrand)]
    result+=[np.real(np.sum(res1[0:-1]*c_n[0:-1]*2.)+res1[-1]*c_n[-1])]
print('Time taken = ', time.time()-begin)
Cl2=chi_cmb * np.real(np.asarray(result))*4./np.pi**2*prefac**2/4.


In [ ]:
# # def Wout(r, xmax):
# #     x1 = r*xmax
# #     return (xmax - x1)/(xmax*x1) * (xmax > x1)

# # def Win(t, r, xmax):
# #     x2 = r*t*xmax
# #     return (xmax - x2)/(xmax*x2) * (xmax > x2)

# def Wkernel(xi, xmax):
#     return (xmax - xi)/(xmax*xi) * (xmax > xi)


# r2d, t2d = np.meshgrid(t_,t_)
# w11, w12 = np.meshgrid(w1,w1)
# begin=time.time()
# result=[]
# lmax = 10
# for ii,ll in enumerate(ell_[:lmax]):
#     if ii%20 ==0: print(ii)
#     res1, res2=[], []
#     for jj, nu in enumerate(nu_n_):
# #         ww = Wout(r2d, chi_cmb) *Win(t2d, r2d, chi_cmb)
# #         integrand=ww*I2D[ii][jj]*D(r2d*t2d*chi_cmb)*D(r2d*chi_cmb)*(r2d*chi_cmb)**(1-nu)
# #         res1+=[np.sum(w11*w12*integrand)]
    
# #         ww = Wout(r2d, chi_cmb) *Win(1/t2d, r2d, chi_cmb)
# #         integrand=ww*I2D[ii][jj]*D(r2d*1/t2d*chi_cmb)*D(r2d*chi_cmb)*(r2d*chi_cmb)**(1-nu) * t2d**(nu-2)
# #         res2+=[np.sum(w11*w12*integrand)]    
# #     r1 = np.real(np.sum(res1[0:-1]*c_n[0:-1]*2.)+res1[-1]*c_n[-1])
# #     r2 = np.real(np.sum(res2[0:-1]*c_n[0:-1]*2.)+res2[-1]*c_n[-1])
# #     result2 += [r1+r2]

#         chi1fac = Wkernel(r2d*chi_cmb, chi_cmb)*D(r2d*chi_cmb)*(r2d*chi_cmb)**(1-nu)
#         chi2fac = Wkernel(t2d*r2d*chi_cmb, chi_cmb)*D(r2d*t2d*chi_cmb) + \
#                         Wkernel(1/t2d*r2d*chi_cmb, chi_cmb)*D(r2d*1/t2d*chi_cmb)* t2d**(nu-2)

#         integrand = chi1fac * chi2fac * I2D[ii][jj]
#         res1+=[np.sum(w11*w12*integrand)]
    
#     result += [np.real(np.sum(res1[0:-1]*c_n[0:-1]*2.)+res1[-1]*c_n[-1])]
# Cl3=chi_cmb * np.real(np.asarray(result))*4./np.pi**2*prefac**2/8.

# print('Time taken = ', time.time()-begin)

In [ ]:
# ### Try with the auto-spectra without symmetrizing so that we check the second term of Eq. 3.05
# ##Also, no need to estimate the kernels again and again since some terms are independent of \ell and \nu.
# ##This way we reduce calls to interpolate and stuff


# def Wkernel(xi, xmax):
#     return (xmax - xi)/(xmax*xi) * (xmax > xi)


# r2d, t2d = np.meshgrid(t_,t_)
# w11, w12 = np.meshgrid(w1,w1)
# begin=time.time()
# result=[]
# lmax = ell_.size 
# chi1fac0 = Wkernel(r2d*chi_cmb, chi_cmb)*D(r2d*chi_cmb)
# chi2fac00 = Wkernel(t2d*r2d*chi_cmb, chi_cmb)*D(r2d*t2d*chi_cmb) 
# chi2fac01 = Wkernel(1/t2d*r2d*chi_cmb, chi_cmb)*D(r2d*1/t2d*chi_cmb)

# for ii,ll in enumerate(ell_[:lmax]):
#     if ii%20 ==0: print(ii)
#     res1, res2=[], []
#     for jj, nu in enumerate(nu_n_):

#         chi1fac = chi1fac0*(r2d*chi_cmb)**(1-nu)
#         chi2fac = chi2fac00 + chi2fac01* t2d**(nu-2)

#         integrand = chi1fac * chi2fac * I2D[ii][jj]
#         res1+=[np.sum(w11*w12*integrand)]
    
#     result += [np.real(np.sum(res1[0:-1]*c_n[0:-1]*2.)+res1[-1]*c_n[-1])]
# Cl3=chi_cmb * np.real(np.asarray(result))*4./np.pi**2*prefac**2/8.

# print('Time taken = ', time.time()-begin)

In [ ]:
### Try with the auto-spectra without symmetrizing so that we check the second term of Eq. 3.05
##Also, no need to estimate the kernels again and again since they are independent of \ell and \nu
##This way we reduce calls to interpolate and stuff
##Well, some things depend on \nu so Use \nu as the third axes so that all the \chi kernels can be estimated beforehand

def Wkernel(xi, xmax):
    return (xmax - xi)/(xmax*xi) * (xmax > xi)


begin=time.time()

r2d, t2d = np.meshgrid(t_,t_)
w11, w12 = np.meshgrid(w1,w1)
I2D2 = np.swapaxes(I2D, 2, 3) #order of 'r' and 't' is confusing to me

result=[]

chi1fac0 = (Wkernel(r2d*chi_cmb, chi_cmb)*D(r2d*chi_cmb)).reshape(*r2d.shape, 1)
chi1fac0 = chi1fac0 *(r2d*chi_cmb).reshape(*r2d.shape, 1)**(1-nu_n_.reshape(1, 1, -1))

chi2fac00 = (Wkernel(t2d*r2d*chi_cmb, chi_cmb)*D(r2d*t2d*chi_cmb)).reshape(*r2d.shape, 1)
chi2fac01 = (Wkernel(1/t2d*r2d*chi_cmb, chi_cmb)*D(r2d*1/t2d*chi_cmb)).reshape(*r2d.shape, 1)
chi2fac01 = chi2fac01 * t2d.reshape(*r2d.shape, 1)**(nu_n_.reshape(1, 1, -1)-2)
chi2fac0 = chi2fac00 + chi2fac01
w11, w12 = w11.reshape(*w11.shape, 1), w12.reshape(*w12.shape, 1)

lmax = ell.size
for ii,ll in enumerate(ell_[:lmax]):
    if ii%20 ==0: print(ii) 
        
    res1 = np.sum(w11*w12*chi1fac0* chi2fac0*I2D2[ii].T, axis=(0, 1))
    
    result += [np.real(np.sum(res1[0:-1]*c_n[0:-1]*2.)+res1[-1]*c_n[-1])]
Cl3=chi_cmb * np.real(np.asarray(result))*4./np.pi**2*prefac**2/8.

print('Time taken = ', time.time()-begin)

In [ ]:
plt.figure()
plt.loglog(ell_,(ell_*(ell_+1.))**2*Clv/(2.*np.pi))
plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]+1.))**2*Cl2[:lmax]/(2.*np.pi))
plt.loglog(ell_[:lmax],(ell_[:lmax]*(ell_[:lmax]+1.))**2*Cl3[:lmax]/(2.*np.pi))
plt.xlim(2,500)

In [ ]:
ll,clpp,clpp2 = np.loadtxt('class_ouput.out')

plt.figure()#
plt.semilogx(ell_,Clv/np.interp(ell_,ll,clpp), label='nessa')
plt.semilogx(ell_,Cl2/np.interp(ell_,ll,clpp), ls="--", lw=2, label='genkern')
plt.semilogx(ell_,Cl3/np.interp(ell_,ll,clpp), ls=":", lw=3, label='mcSpeedy')
# plt.semilogx(ell_[:lmax], Cl2[:lmax]/np.interp(ell_[:lmax],ll,clpp), ls="--", label='our result over CLASS no Limber')
plt.ylim(0.95,1.05)
# plt.ylim(0.9,2)
plt.grid()
plt.legend()